In [1]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
# This line is only needed if you have a HiDPI display
%config InlineBackend.figure_format = 'retina'

from sklearn.datasets import make_blobs, make_circles, make_moons
from sklearn.preprocessing import StandardScaler
import os, struct
from array import array
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from SVM import SVM
kwargs = {'linewidth' : 3.5}
font = {'weight' : 'normal', 'size'   : 24}
matplotlib.rc('font', **font)

In [2]:
!pip install numpy
!pip install matplotlib
!pip install sklearn

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
def normalize_stats_image_by_image(images):
    """Normalizing images"""
    mean = images.mean(axis=(1,2), keepdims=True)
    stdev = images.std(axis=(1,2), keepdims=True)
    return (images - mean) / stdev

def acc_plot(w, img_vectors, labels, dilate=1):
    """plot accuracies"""
    acc_list = cal_acc_dilate(w, img_vectors, labels,  dilate)
    print(acc_list[0])
    plt.xlabel('Step')
    plt.ylabel('Acc')
    num_iters = len(acc_list)
    plt.plot(np.linspace(0, num_iters*dilate, num=num_iters), acc_list, **kwargs)

def cal_acc_dilate(w, img_vectors, labels, dilate=1):
    num_iters = len(w)
    return [compute_accuracy(x, img_vectors, labels) for x in w[0:num_iters:dilate]]

def compute_accuracy(w, imgs, labels):
    """Compute the accuracy"""
    correct = 0
    for i in range(0, imgs.shape[0]):
        pred = w.T @ imgs[i].T>=0
        if pred==0:
            pred=-1
        correct += int(labels[i]==pred)
    acc = correct/imgs.shape[0]
    return acc

In [4]:
import gzip
#FROM: https://gist.github.com/tylerneylon/ce60e8a06e7506ac45788443f7269e40
def read_idx(filename):
    with gzip.open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
test_labels = read_idx('t10k-labels-idx1-ubyte.gz')
test_imgs = read_idx('t10k-images-idx3-ubyte.gz')
train_imgs = read_idx('train-images-idx3-ubyte.gz')
train_labels = read_idx('train-labels-idx1-ubyte.gz') 

mask_6_8 = (train_labels == 6) | (train_labels == 8)
X_train_scaled1 = normalize_stats_image_by_image(train_imgs[mask_6_8])
y = (train_labels[mask_6_8] == 8).astype(np.int) * 2 -1
y = y[:3000]
X_train_scaled = X_train_scaled1.reshape(11769, -1)[:3000, :]
# X_train_scaled = X_train_scaled1.reshape(11769, -1)
mask_test = (test_labels == 6) | (test_labels == 8)
test_imagess = test_imgs[mask_test]
# print(test_imagess.shape)
test_labels_6_8 = (test_labels[mask_test] == 8).astype(np.int) * 2 -1
test_images = test_imagess.reshape(1932, -1)

FileNotFoundError: [Errno 2] No such file or directory: 't10k-labels-idx1-ubyte.gz'

In [ ]:
error_ls  = []
model = SVM(max_iter=1, kernel_type='linear', C=1.0, epsilon=0.001)
model.fit(X_train_scaled, y)
print("kuku")
for i in range(len(model.w_ls)):
    if i % 10 == 0:
#         e = 1 - (X_train_scaled @ model.w_ls[i]) * y
#         error_ls.append(np.linalg.norm(model.w_ls[i])**2 / 2.0 + np.mean((e > 0) * e))
        error_ls.append(np.average(abs(model.E(X_train_scaled, y, model.w_ls[i], model.b_ls[i]))))
#         print("heihei")
plt.plot(error_ls)

In [ ]:
print(error_ls[-1])
plt.plot(error_ls[:300])

In [ ]:
e_ls = []
for i in range(len(model.w_ls)):
    if i % 10 == 0:
        e = 1 - (X_train_scaled @ model.w_ls[i]) * y
        e_ls.append(np.linalg.norm(model.w_ls[i])**2 / 2.0 + np.mean((e > 0) * e))
#         print(np.mean((e > 0) * e))
#         error_ls.append(np.average(abs(model.E(X_train_scaled, y, model.w_ls[i], model.b_ls[i]))))
#         print("heihei")
plt.plot(e_ls)

In [ ]:
acc_plot(model.w_ls, X_train_scaled, y, 10)

In [ ]:
print(np.linalg.norm(model.w_ls[-1]))